<h1> UCI - I&C SCI_X426.53 (Fall2017) </h1>
**Course Instructors:** <br>
Ted Pham (email:tedp@uci.edu) <br>
William Henry  (will@henryanalytics.com)



<h2>Week7 Assignment</h2>


---
This is an individual assignment, but feel free to exchange ideas on the forum.
__Name:__  Your Name Here<br>
__Email:__  peter_anteater@uci.edu   
__Week:__   7

__Due Time:__ Friday Dec 21 10am PST

# Overview

In this notebook, we will go over examples of running Spark and do some exercises
    1. KDnuggets Tutorial
    2. WordCount Exercise
    3. K-means Example
    4. K-means Exercise

This notebook was tested on AWS EC2 jupyter interface using UCI AMI provided in previous weeks.



# 0. Initialize Spark

In [2]:
#Import SparkContext from pyspark
from pyspark import SparkContext
sc = SparkContext()

In [3]:
# Try Week6 code
from operator import add
rdd1 = sc.parallelize([("a",1),("b",1),("a",1)])
sorted(rdd1.reduceByKey(add).collect())


[('a', 2), ('b', 1)]

# 1. KDnuggets Spark Tutorial

kdnuggets.com has an excellent tutorial on Spark and Python
https://www.kdnuggets.com/2015/11/introduction-spark-python.html

Due to copyrights, please read the tutorial and we follow along a slightly modified code here.

Starting with downloading the daily_show_guests.csv. 


In [9]:
!curl -L https://github.com/fivethirtyeight/data/blob/master/daily-show-guests/daily_show_guests.csv -o daily.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  749k    0  749k    0     0   749k      0 --:--:-- --:--:-- --:--:-- 2507k


In [1]:
!head -10 daily.csv

YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List
1999,actor,1/11/99,Acting,Michael J. Fox
1999,Comedian,1/12/99,Comedy,Sandra Bernhard
1999,television actress,1/13/99,Acting,Tracey Ullman
1999,film actress,1/14/99,Acting,Gillian Anderson
1999,actor,1/18/99,Acting,David Alan Grier
1999,actor,1/19/99,Acting,William Baldwin
1999,Singer-lyricist,1/20/99,Musician,Michael Stipe
1999,model,1/21/99,Media,Carmen Electra
1999,actor,1/25/99,Acting,Matthew Lillard


In [4]:
#-get the first 5 elements from the raw RDD
raw = sc.textFile("daily.csv")
raw.take(5)

['YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List',
 '1999,actor,1/11/99,Acting,Michael J. Fox',
 '1999,Comedian,1/12/99,Comedy,Sandra Bernhard',
 '1999,television actress,1/13/99,Acting,Tracey Ullman',
 '1999,film actress,1/14/99,Acting,Gillian Anderson']

In [5]:
#-split elements by commas
daily = raw.map(lambda line: line.split(','))
daily.take(5)

[['YEAR', 'GoogleKnowlege_Occupation', 'Show', 'Group', 'Raw_Guest_List'],
 ['1999', 'actor', '1/11/99', 'Acting', 'Michael J. Fox'],
 ['1999', 'Comedian', '1/12/99', 'Comedy', 'Sandra Bernhard'],
 ['1999', 'television actress', '1/13/99', 'Acting', 'Tracey Ullman'],
 ['1999', 'film actress', '1/14/99', 'Acting', 'Gillian Anderson']]

In [6]:
# Aggregate total count of visitors per year
tally = daily.map(lambda x: (x[0], 1))\
             .reduceByKey(lambda x,y: x+y)
print(tally)

PythonRDD[14] at RDD at PythonRDD.scala:48


In [7]:
# because Spark is lazy we need to perform an action on the RDD
tally.take(tally.count())

[('YEAR', 1),
 ('2002', 159),
 ('2003', 166),
 ('2004', 164),
 ('2007', 141),
 ('2010', 165),
 ('2011', 163),
 ('2012', 164),
 ('2013', 166),
 ('2014', 163),
 ('2015', 100),
 ('1999', 166),
 ('2000', 169),
 ('2001', 157),
 ('2005', 162),
 ('2006', 161),
 ('2008', 164),
 ('2009', 163)]

**Question 1 (5pts)**
Sort the tally by year

In [ ]:
#Student Code#


In [11]:
#Filter out empty profession, make profession lowercase
# and get profession count for the visitors
filtered_daily = daily.filter(lambda line: line[1] != '') \
                      .map(lambda line: (line[1].lower(), 1))\
                      .reduceByKey(lambda x,y: x+y)
        
# get first 5 profession counts
filtered_daily.take(5)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271)]

**Question 2 (5pts)** Get all professions and counts


In [ ]:
#-Student Code-#



# 2. WordCount Exercise

In [12]:
# We will perform word count using Spark with Christmas Carol Text
# If you are working with the notebook on your local computer instead 
# of EC2 you can download the text from https://www.gutenberg.org/files/46/46-h/46-h.htm
# We look at first 10 lines
!head -10 christmas.txt

A CHRISTMAS CAROL

IN PROSE
BEING
A Ghost Story of Christmas

by Charles Dickens





**Total Word Count from Christmas Carol***

In [15]:
# Create a textfile RDD
christmas = sc.textFile("christmas.txt")

#simple actions on the RDD
print(christmas.count())
print(christmas.first())
print(christmas.take(10))


4208
A CHRISTMAS CAROL
['A CHRISTMAS CAROL', '', 'IN PROSE', 'BEING', 'A Ghost Story of Christmas', '', 'by Charles Dickens', '', '', '']


**Question 3 (10 pts)**
Use Spark, get the 20 most common "lowercased" words (don't count stopwords)

Hint: Pseudocode

1. Define a function "findWord" that takes a line as input and return the words & their counts (if stopwords, don't count the word)

2. define a count RDD as:
    a. flatmap(find,Word)
    b. aggregate by key, add count
    c. switch ke

(1745, 'the')
(1128, 'and')
(775, 'of')
(764, 'a')
(759, 'to')
(577, 'in')
(557, 'it')
(496, 'he')
(428, 'was')
(420, 'his')
(367, 'i')
(365, 'that')
(362, 'scrooge')
(314, 'with')
(314, 'you')
(246, 's')
(239, 'as')
(222, 'for')
(221, 'said')
(205, 'had')


**Question 4 (10 pts)**
Use Spark, get the 20 least common "lowercased" words (don't count stopwords)